In [81]:
import requests
import re
import pandas as pd
import io
import html5lib
from bs4 import BeautifulSoup as bs
from IPython.display import clear_output
import platform

Any urls we may need

In [82]:
_BASE_URL = 'https://verd.lyfjastofnun.is/'
_URL = 'https://verd.lyfjastofnun.is/?pageid=10'

Fetch the urls of all excel files, do not get xlsx files that were uploaded due to them being a security issue.

https://stackoverflow.com/questions/65250207/pandas-cannot-open-an-excel-xlsx-file

In [83]:
r = requests.get(_URL)
soup = bs(r.text)
#xlsx
_links = soup.find_all('a', {'href': re.compile(r'verdskra.*\.xls$')})

Iterate through all excel files and add to our dataframe for processing.

Skip the first file as it simply explains the columns of the rest of the files.

In [84]:
appended_data = []
for index, link in enumerate(_links[1:]):
    price_link = link.get("href")
    file_name = re.search('gogn/(.*).xls',price_link).group(1)
    response = requests.get(_BASE_URL + price_link).content
    inc_data = pd.read_excel(response).assign(file=file_name)
    appended_data.append(inc_data)

data = pd.concat(appended_data)
data.head(10)

,Norrænt vörunúmer,Lyfjaheitisnúmer,Heiti lyfs,Form lyfs,Styrkur,Styrkeining,Magn,Magneining,Fj. Pakkninga,Pakkning,...,"Skammtur að hámarki fyrir fjölda daga (100, 30)",Heildsöluverð,Smásöluverð,Meðferð,Hmagn,"Á undanþágulista (1, 0)",DDD,Unnamed: 1,Afskáningardags,Unnamed: 36
0,379875.0,N05BA0105,Stesolid,eþ-lausn,5,"mg/2,5ml","2,5",ml,5.0,túpa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
1,379883.0,N05BA0105,Stesolid,eþ-lausn,10,"mg/2,5ml","2,5",ml,5.0,túpa,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
2,43729.0,N05CD0503,Halcion,töflur,"0,125",mg,30,stk,1.0,þpakki,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
3,48199.0,A03CA0201,Librax,töflur,NaN,NaN,100,stk,1.0,glas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
4,52286.0,N05BA1205,Alprazolam Mylan,töflur,"0,25",mg,20,stk,1.0,þpakki,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
5,52297.0,N05BA1205,Alprazolam Mylan,töflur,"0,25",mg,50,stk,1.0,þpakki,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
6,52309.0,N05BA1205,Alprazolam Mylan,töflur,"0,5",mg,20,stk,1.0,þpakki,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
7,52320.0,N05BA1205,Alprazolam Mylan,töflur,"0,5",mg,50,stk,1.0,þpakki,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
8,52331.0,N05BA1205,Alprazolam Mylan,töflur,"0,25",mg,100,stk,1.0,glas,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN
9,70203.0,N05CF0107,Imomed,filmhtfl,"7,5",mg,30,stk,1.0,þpakki,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN


Time to clean the data as best we can. Lets merge all the data.

In [88]:
data['nvnr'] = data['Norrænt vörunúmer'].combine_first(data['Nvnr'])
data['form'] = data['Form lyfs'].combine_first(data['Form'])
data['package_quantity'] = data['Fj. Pakkninga'].combine_first(data['Fjöldi pakkninga'])
data = data.drop(columns=['Norrænt vörunúmer','Nvnr','Form lyfs','Form','Fj. Pakkninga','Fjöldi pakkninga'])
# data['nvnr'] = data['Norrænt vörunúmer'].combine_first(data['Nvnr'])
# data['nvnr'] = data['Norrænt vörunúmer'].combine_first(data['Nvnr'])
# data['nvnr'] = data['Norrænt vörunúmer'].combine_first(data['Nvnr'])
# data['nvnr'] = data['Norrænt vörunúmer'].combine_first(data['Nvnr'])
# data['nvnr'] = data['Norrænt vörunúmer'].combine_first(data['Nvnr'])

In [89]:
data.head(10)

,Lyfjaheitisnúmer,Heiti lyfs,Styrkur,Styrkeining,Magn,Magneining,Pakkning,Athugasemdir,ATC-flokkur,Skráningardagsetning,...,Meðferð,Hmagn,"Á undanþágulista (1, 0)",DDD,Unnamed: 1,Afskáningardags,Unnamed: 36,nvnr,form,package_quantity
0,N05BA0105,Stesolid,5,"mg/2,5ml","2,5",ml,túpa,NaN,N05BA01,1.10.1995,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,379875.0,eþ-lausn,5.0
1,N05BA0105,Stesolid,10,"mg/2,5ml","2,5",ml,túpa,NaN,N05BA01,1.10.1995,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,379883.0,eþ-lausn,5.0
2,N05CD0503,Halcion,"0,125",mg,30,stk,þpakki,NaN,N05CD05,1.12.1983,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,43729.0,töflur,1.0
3,A03CA0201,Librax,NaN,NaN,100,stk,glas,NaN,A03CA02,1.12.1972,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,48199.0,töflur,1.0
4,N05BA1205,Alprazolam Mylan,"0,25",mg,20,stk,þpakki,NaN,N05BA12,1.10.2009,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,52286.0,töflur,1.0
5,N05BA1205,Alprazolam Mylan,"0,25",mg,50,stk,þpakki,NaN,N05BA12,1.10.2009,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,52297.0,töflur,1.0
6,N05BA1205,Alprazolam Mylan,"0,5",mg,20,stk,þpakki,NaN,N05BA12,1.7.2009,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,52309.0,töflur,1.0
7,N05BA1205,Alprazolam Mylan,"0,5",mg,50,stk,þpakki,NaN,N05BA12,1.10.2009,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,52320.0,töflur,1.0
8,N05BA1205,Alprazolam Mylan,"0,25",mg,100,stk,glas,NaN,N05BA12,1.8.2009,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,52331.0,töflur,1.0
9,N05CF0107,Imomed,"7,5",mg,30,stk,þpakki,NaN,N05CF01,1.5.2020,...,NaN,NaN,NaN,NaN,NaN,NaT,NaN,70203.0,filmhtfl,1.0


In [90]:
data.to_csv("output.txt")